In [1]:
NSIMSAMPS = 100_000    # Samples to draw from the simulator
epsilon = 0.1          # epsilon-greedy behavior policy
runs = list(range(10)) # Repeat for 10 replications

output_dir = '../datagen/eps_0_1-100k-isd/'

# Features

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from tqdm import tqdm
from collections import defaultdict
import pickle
import itertools
import copy
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import joblib
from joblib import Parallel, delayed

In [4]:
import scipy.sparse

## 21-dimensional state features

In [5]:
nS, nA = 1442, 8
d = 21

In [6]:
def get_state_action_feature(x_s, a):
    x_sa = np.zeros((nA, d))
    x_sa[a, :] = x_s
    return x_sa.flatten()

In [7]:
def make_features_single_trajectory(df_i):
    # Initial timestep
    s_init = df_i.iloc[0, 7:28].values
    x_s_init = np.array(s_init)
    xa_s_init_all = np.array([get_state_action_feature(x_s_init, a_) for a_ in range(nA)])

    # Intermediate timestep
    if len(df_i) > 1:
        s = df_i.iloc[:-1, 7:28].values
        a = df_i.iloc[:-1]['Action'].values
        r = df_i.iloc[:-1]['Reward'].values
        s_next = df_i.iloc[1:, 7:28].values

        n = len(s)
        x_s = np.array(s)
        xa_sa = np.array([get_state_action_feature(x_s[j, :], a[j]) for j in range(n)])

        x_s_next = np.array(s_next)
        xa_s_next_all = np.vstack([
            np.vstack([get_state_action_feature(x_s_next[j], a_) for a_ in range(nA)]) 
            for j in range(n)
        ])
    else:
        x_s = np.array((0, d))
        a = np.zeros((0), dtype=int)
        xa_sa = np.array((0, d*nA))
        r = np.zeros((0))
        x_s_next = np.array((0, d))
        xa_s_next_all = np.array((0, d*nA))

    # Final timestep
    s_last = df_i.iloc[-1, 7:28].values
    a_last = df_i.iloc[-1]['Action']
    r_last = df_i.iloc[-1]['Reward']
    if r_last == -1 or r_last == 1:
        # Reached death/disch states
        # every action leads to reward
        x_s_last = np.array(s_last)
        xa_s_last_all = np.array([get_state_action_feature(x_s_last, a_) for a_ in range(nA)])
        r_last_all = np.array(nA * [r_last])

        xa_out = np.vstack([xa_sa, xa_s_last_all])
        xa_next_out = np.vstack([xa_s_next_all, np.zeros((nA*nA, nA*d))])
        r_out = np.concatenate([r, r_last_all])

        a_out = np.concatenate([a, (list(range(nA)))])
        x_out = np.vstack([x_s, *(nA*[x_s_last])])
        x_next_out = np.vstack([x_s_next, np.zeros((nA, d))])
    else: 
        # terminated early due to max length, so no next state information
        xa_out = xa_sa
        xa_next_out = xa_s_next_all
        r_out = r

        x_out = x_s
        a_out = a
        x_next_out = x_s_next
    
    return x_s_init, xa_s_init_all, x_out, a_out, xa_out, r_out, x_next_out, xa_next_out

In [8]:
for it in runs:
    df_features = pd.read_csv('{}/{}-features.csv'.format(output_dir, it))
    out = [make_features_single_trajectory(df_i) for i, df_i in tqdm(df_features.groupby('pt_id'))]
    X_init, Xa_init, X, A, Xa, R, X_next, Xa_next = zip(*out)
    X_init = np.vstack(X_init)
    Xa_init = np.vstack(Xa_init)
    X = np.vstack(X)
    Xa = np.vstack(Xa)
    A = np.concatenate(A)
    R = np.concatenate(R)
    X_next = np.vstack(X_next)
    Xa_next = np.vstack(Xa_next)
    print(Xa_init.shape, Xa.shape, Xa_next.shape, R.shape)
    print(X_init.shape, X.shape, A.shape, R.shape, X_next.shape)

    # Store indices of beginning of each episode
    lengths = [len(x_i) for x_i in list(zip(*out))[2]]
    inds_init = np.cumsum([0] + lengths)

    joblib.dump({
        'X_init': X_init, 'X': X, 'A': A, 'R': R, 'X_next': X_next, 
        'Xa_init': Xa_init, 'Xa': Xa, 'Xa_next': Xa_next,
        'lengths': lengths, 'inds_init': inds_init,
    }, '{}/{}-21d-feature-matrices.joblib'.format(output_dir, it))

    joblib.dump({
        'X_init': scipy.sparse.csr_matrix(X_init), 'X': scipy.sparse.csr_matrix(X), 'A': A, 'R': R, 'X_next': scipy.sparse.csr_matrix(X_next), 
        'Xa_init': scipy.sparse.csr_matrix(Xa_init), 'Xa': scipy.sparse.csr_matrix(Xa), 'Xa_next': scipy.sparse.csr_matrix(Xa_next),
        'lengths': lengths, 'inds_init': inds_init,
    }, '{}/{}-21d-feature-matrices.sparse.joblib'.format(output_dir, it))

100%|██████████| 100000/100000 [03:12<00:00, 518.70it/s]


(800000, 168) (1689110, 168) (13512880, 168) (1689110,)
(100000, 21) (1689110, 21) (1689110,) (1689110,) (1689110, 21)


100%|██████████| 100000/100000 [03:13<00:00, 515.98it/s]


(800000, 168) (1685108, 168) (13480864, 168) (1685108,)
(100000, 21) (1685108, 21) (1685108,) (1685108,) (1685108, 21)


100%|██████████| 100000/100000 [03:07<00:00, 532.48it/s]


...

(800000, 168) (1687678, 168) (13501424, 168) (1687678,)
(100000, 21) (1687678, 21) (1687678,) (1687678,) (1687678, 21)


100%|██████████| 100000/100000 [03:09<00:00, 526.86it/s]


(800000, 168) (1685824, 168) (13486592, 168) (1685824,)
(100000, 21) (1685824, 21) (1685824,) (1685824,) (1685824, 21)


100%|██████████| 100000/100000 [03:10<00:00, 524.62it/s]


(800000, 168) (1688402, 168) (13507216, 168) (1688402,)
(100000, 21) (1688402, 21) (1688402,) (1688402,) (1688402, 21)
